In [1]:
import time, os, sys
os.environ['OMP_NUM_THREADS']="1"
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['OMP_NUM_THREADS']='1'
os.environ['MKL_NUM_THREADS']='1'

In [2]:
import numpy as np
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
from cellpose import utils, io
import glob

In [3]:
from cellpose import models, io

In [4]:
dapi_dir = '/projects/ldr139/spatial_data/output0/*.tif'

In [ ]:
images = glob.glob(dapi_dir)
# Looping through list to find "Brain" index and retrieving 6th to 11th string from there which should be brain A-H underscore 1-6 + A/B/AB
# Looping through list to find "Brain" index and retrieving 12th to the 4th last string which should be the channel name
section = []
for b in range(len(images)):
    section.append((images[b][(images[b].find("Brain") + 6):(images[b].find("Brain") + 11)]).rstrip("_"))
mice = list(dict.fromkeys(section))

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(io.imread(images[0]))
plt.axis('off')
plt.show()

In [ ]:
img_dict = {}
for image in images:
    for sect in mice:
        if sect in image:
            img_dict[sect] = image

In [ ]:
from cellpose import models, io
model = models.Cellpose(gpu=True, model_type='cyto2')
channels = [[0,0]]

import time
start_time = time.time()

for section in img_dict.keys():
    print(section)
    img = io.imread(img_dict[section])
    masks, flows, styles, diams = model.eval(img, diameter=70, channels=channels)
    io.masks_flows_to_seg(img, masks, flows, diams, 'dapi_segs/'+section+'_DAPI', channels)
    io.save_to_png(img, masks, flows, 'dapi_segs/'+section+'_DAPI')
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
!pwd

In [ ]:
mask_dir = {}
mask_arrays = glob.glob("dapi_segs/*.npy")
for sect in mice:
    if "dapi_segs/"+sect+"_DAPI_seg.npy" in mask_arrays:
        mask_dir[sect]=np.load("dapi_segs/"+sect+"_DAPI_seg.npy", allow_pickle=True).item()["masks"]

In [ ]:
for key in mask_dir.keys():
    print(key, mask_dir[key].shape, mask_dir[key].max())

In [ ]:
plt.imshow(np.load("dapi_segs/G_1A_DAPI_seg.npy", allow_pickle=True).item()['masks'])

In [ ]:
!pwd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(np.load("dapi_segs/A_1AB_DAPI_seg.npy", allow_pickle=True).item()['masks'])
ax.axis('off')

plt.show()

In [ ]:
!pwd